In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Input,Conv2D,MaxPooling2D,GlobalAveragePooling2D,Dense,SpatialDropout1D,Flatten,Conv1D,LSTM,GlobalAveragePooling1D
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import librosa
from keras.layers import *
import os
import keras
from keras.models import Model
print(os.listdir("../input"))
from keras.utils import to_categorical
from glob import glob
from tqdm import tqdm
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import KFold
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['audio_speech_actors_01-24']


In [2]:
audiodir=glob("../input/audio_speech_actors_01-24/*")

In [3]:
audiodirfilename=[glob(dir1+"/*") for dir1 in audiodir]

In [4]:
audiodirfilename1=[]
for i in audiodirfilename:
    audiodirfilename1.extend(i)

In [5]:
len(audiodirfilename1)

1440

In [6]:
sr=26000
duration=4.5
hop_length=400
fmin=20
fmax=sr//2
n_mels=128
n_fft=n_mels*20
samples=int(sr*duration)


In [7]:
def loadaudio(path):
    y,sr1=librosa.core.load(path,sr)
    #print(sr1)
    if len(y)<samples:
        length=samples-len(y)
        y=np.pad(y,(0,length),'constant')
    else:
        y=y[0:samples]
    return y

def audio_melspectrogram(x):
    spectrogram=librosa.feature.melspectrogram(x,sr=sr,n_mels=n_mels,n_fft=n_fft,fmin=fmin,fmax=fmax)
    return librosa.power_to_db(spectrogram).astype(np.float32)

def convert_wav_image(paths):
    X=[]
    for path in tqdm(paths):
        audio=loadaudio(path)
        spec=audio_melspectrogram(audio)
        X.append(spec.transpose())
    return X

def normalize(img):
    
    return img

def normalize_dataset(X):
    '''
    Normalizes list of arrays
    (subtract mean and divide by standard deviation)
    '''
    normalized_dataset = []
    for img in X:
        normalized = normalize(img)
        normalized_dataset.append(normalized)
    normalized_dataset=np.array(normalized_dataset)    
    eps=0.01
    
    normalized_dataset = (normalized_dataset - np.mean(normalized_dataset,axis=0)) / np.std(normalized_dataset,axis=0)
    
    return normalized_dataset

In [8]:
y=librosa.feature.melspectrogram(loadaudio(audiodirfilename1[0]),sr=sr,n_mels=n_mels,n_fft=n_fft,fmin=fmin,fmax=fmax)

In [9]:
import IPython.display as ipd  # To play sound in the notebook
fname = audiodirfilename1[802]   # Raindrop
ipd.Audio(fname)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = np.array(convert_wav_image(audiodirfilename1))
X = normalize_dataset(X)

100%|██████████| 1440/1440 [04:09<00:00,  5.88it/s]


In [12]:
ytrain=[int(x.split("/")[-1].split("-")[2][1]) for x in audiodirfilename1]

In [13]:
ytrain1=to_categorical(np.array(ytrain)-1,8) 

In [14]:
X1=np.array(X)

In [15]:
len(X1)

1440

In [16]:
kfold=KFold(6,random_state=42)

In [17]:

def spatial_squeeze_excite_block(input1):
    se=Conv1D(1,(1),activation='sigmoid',kernel_initializer='he_normal')(input1)
    x=Multiply()([input1,se])
    return x

def squeeze_excite_block(input1,ratio=4):
    se=GlobalAveragePooling1D()(input1)
    se=Dense(input1._keras_shape[-1]//ratio,activation='relu')(se)
    se=Dense(input1._keras_shape[-1],activation='sigmoid')(se)
    se=Reshape((1,input1._keras_shape[-1]))(se)
    finput=Multiply()([input1,se])
    return finput

def spatial_Csqueeze_excite_block(input1,ratio=4):
    squeezeblock=squeeze_excite_block(input1,ratio)
    spatialblock=spatial_squeeze_excite_block(input1)
    finalblock=Add()([squeezeblock,spatialblock])
    return finalblock

In [18]:
def model(trainx,trainy,testx,testy):
    inp = keras.Input((229,128))

# conv_1
    conv1 = Conv1D(filters=32, kernel_size=4, strides=1, padding='same')(inp)
    conv1 = keras.layers.SpatialDropout1D(.4)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = spatial_Csqueeze_excite_block(conv1)

    conv2 = Conv1D(filters=32, kernel_size=6, strides=1, padding='same')(inp)
    conv2 = keras.layers.SpatialDropout1D(.4)(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = spatial_Csqueeze_excite_block(conv2)

    conv3 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(inp)
    conv3 = keras.layers.SpatialDropout1D(.4)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = spatial_Csqueeze_excite_block(conv3)
    concat= Concatenate(axis=-1)([conv1,conv2,conv3])


    conv1 = Conv1D(filters=32, kernel_size=4, strides=2, padding='same')(concat)
    conv1 = keras.layers.SpatialDropout1D(.4)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = spatial_Csqueeze_excite_block(conv1)

    conv2 = Conv1D(filters=32, kernel_size=6, strides=2, padding='same')(concat)
    conv2 = keras.layers.SpatialDropout1D(.4)(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = spatial_Csqueeze_excite_block(conv2)

    conv3 = Conv1D(filters=32, kernel_size=5, strides=2, padding='same')(concat)
    conv3 = keras.layers.SpatialDropout1D(.4)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = spatial_Csqueeze_excite_block(conv3)


    concat= Concatenate(axis=-1)([conv1,conv2,conv3])

    laststate=LSTM(256,dropout=0.16,recurrent_dropout=0.16)(concat)
    out=Dense(8,activation='softmax')(laststate)
    model=Model(inp,out)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(trainx,trainy,batch_size=32,epochs=250,verbose=2,shuffle=True,validation_split=0.1)
    
    _,testacc=model.evaluate(testx,testy)
    
    return model,testacc

In [19]:
models=[]
testaccuracies=[]

for trainidx,testidx in kfold.split(X1):
    trainxbatch=X1[trainidx]
    trainybatch=ytrain1[trainidx]
    testxbatch=X1[testidx]
    testybatch=ytrain1[testidx]
    model1,testacc=model(trainxbatch,trainybatch,testxbatch,testybatch)
    
    models.append(model1)
    testaccuracies.append(testacc)

Train on 1080 samples, validate on 120 samples
Epoch 1/250
 - 20s - loss: 1.9161 - acc: 0.2491 - val_loss: 1.9191 - val_acc: 0.2417
Epoch 2/250
 - 10s - loss: 1.8576 - acc: 0.2435 - val_loss: 1.8460 - val_acc: 0.2083
Epoch 3/250
 - 10s - loss: 1.8337 - acc: 0.2704 - val_loss: 1.8229 - val_acc: 0.2917
Epoch 4/250
 - 10s - loss: 1.8293 - acc: 0.2722 - val_loss: 1.8278 - val_acc: 0.2667
Epoch 5/250
 - 10s - loss: 1.8009 - acc: 0.2824 - val_loss: 1.8135 - val_acc: 0.3083
Epoch 6/250
 - 10s - loss: 1.8222 - acc: 0.2796 - val_loss: 1.8317 - val_acc: 0.2833
Epoch 7/250
 - 11s - loss: 1.8019 - acc: 0.2704 - val_loss: 1.7947 - val_acc: 0.3167
Epoch 8/250
 - 11s - loss: 1.7920 - acc: 0.2759 - val_loss: 1.8253 - val_acc: 0.3167
Epoch 9/250
 - 10s - loss: 1.7933 - acc: 0.2806 - val_loss: 1.8040 - val_acc: 0.3250
Epoch 10/250
 - 10s - loss: 1.7904 - acc: 0.2843 - val_loss: 1.7952 - val_acc: 0.3583
Epoch 11/250
 - 10s - loss: 1.7991 - acc: 0.2954 - val_loss: 1.7900 - val_acc: 0.3333
Epoch 12/250
 - 

Epoch 96/250
 - 10s - loss: 1.3313 - acc: 0.4796 - val_loss: 1.8577 - val_acc: 0.3417
Epoch 97/250
 - 10s - loss: 1.3678 - acc: 0.4676 - val_loss: 1.8925 - val_acc: 0.3500
Epoch 98/250
 - 10s - loss: 1.3809 - acc: 0.4472 - val_loss: 1.7716 - val_acc: 0.3083
Epoch 99/250
 - 10s - loss: 1.3500 - acc: 0.4796 - val_loss: 2.0259 - val_acc: 0.3167
Epoch 100/250
 - 10s - loss: 1.3474 - acc: 0.4611 - val_loss: 1.8117 - val_acc: 0.3083
Epoch 101/250
 - 10s - loss: 1.3717 - acc: 0.4731 - val_loss: 1.8782 - val_acc: 0.3500
Epoch 102/250
 - 10s - loss: 1.3093 - acc: 0.4907 - val_loss: 2.0114 - val_acc: 0.3333
Epoch 103/250
 - 11s - loss: 1.3295 - acc: 0.4750 - val_loss: 1.9041 - val_acc: 0.3583
Epoch 104/250
 - 10s - loss: 1.3013 - acc: 0.5009 - val_loss: 1.9123 - val_acc: 0.2917
Epoch 105/250
 - 10s - loss: 1.3323 - acc: 0.4889 - val_loss: 1.8372 - val_acc: 0.3083
Epoch 106/250
 - 10s - loss: 1.3290 - acc: 0.4944 - val_loss: 1.9960 - val_acc: 0.3333
Epoch 107/250
 - 10s - loss: 1.2941 - acc: 0.49

Epoch 191/250
 - 10s - loss: 0.9028 - acc: 0.6537 - val_loss: 2.2561 - val_acc: 0.3083
Epoch 192/250
 - 11s - loss: 0.8572 - acc: 0.6694 - val_loss: 2.1202 - val_acc: 0.3667
Epoch 193/250
 - 10s - loss: 0.8554 - acc: 0.6907 - val_loss: 2.2886 - val_acc: 0.3333
Epoch 194/250
 - 10s - loss: 0.8554 - acc: 0.6731 - val_loss: 2.2342 - val_acc: 0.3583
Epoch 195/250
 - 10s - loss: 0.8030 - acc: 0.7000 - val_loss: 2.2455 - val_acc: 0.3583
Epoch 196/250
 - 10s - loss: 0.8492 - acc: 0.6861 - val_loss: 2.3006 - val_acc: 0.3500
Epoch 197/250
 - 10s - loss: 0.8144 - acc: 0.6898 - val_loss: 2.3461 - val_acc: 0.3583
Epoch 198/250
 - 10s - loss: 0.8004 - acc: 0.7111 - val_loss: 2.2912 - val_acc: 0.3917
Epoch 199/250
 - 10s - loss: 0.8226 - acc: 0.6889 - val_loss: 2.3486 - val_acc: 0.3667
Epoch 200/250
 - 10s - loss: 0.7861 - acc: 0.7194 - val_loss: 2.4584 - val_acc: 0.3167
Epoch 201/250
 - 10s - loss: 0.7820 - acc: 0.7157 - val_loss: 2.4660 - val_acc: 0.3500
Epoch 202/250
 - 10s - loss: 0.7745 - acc: 

Epoch 35/250
 - 10s - loss: 1.6301 - acc: 0.3778 - val_loss: 1.7107 - val_acc: 0.4000
Epoch 36/250
 - 10s - loss: 1.6221 - acc: 0.3759 - val_loss: 1.7255 - val_acc: 0.3500
Epoch 37/250
 - 10s - loss: 1.5941 - acc: 0.3907 - val_loss: 1.8145 - val_acc: 0.3750
Epoch 38/250
 - 10s - loss: 1.5959 - acc: 0.3778 - val_loss: 1.8236 - val_acc: 0.3667
Epoch 39/250
 - 10s - loss: 1.5956 - acc: 0.3898 - val_loss: 1.7919 - val_acc: 0.3833
Epoch 40/250
 - 10s - loss: 1.5684 - acc: 0.4074 - val_loss: 1.8317 - val_acc: 0.3333
Epoch 41/250
 - 10s - loss: 1.5847 - acc: 0.4083 - val_loss: 1.7912 - val_acc: 0.3750
Epoch 42/250
 - 10s - loss: 1.5748 - acc: 0.4000 - val_loss: 1.7952 - val_acc: 0.3833
Epoch 43/250
 - 10s - loss: 1.5967 - acc: 0.3815 - val_loss: 1.8345 - val_acc: 0.3417
Epoch 44/250
 - 10s - loss: 1.5967 - acc: 0.3935 - val_loss: 1.8029 - val_acc: 0.3917
Epoch 45/250
 - 10s - loss: 1.5592 - acc: 0.3907 - val_loss: 1.7783 - val_acc: 0.3500
Epoch 46/250
 - 10s - loss: 1.5819 - acc: 0.3935 - val

Epoch 130/250
 - 10s - loss: 1.1487 - acc: 0.5519 - val_loss: 1.8832 - val_acc: 0.4083
Epoch 131/250
 - 10s - loss: 1.1376 - acc: 0.5769 - val_loss: 1.9673 - val_acc: 0.4167
Epoch 132/250
 - 10s - loss: 1.1044 - acc: 0.5907 - val_loss: 2.0031 - val_acc: 0.4333
Epoch 133/250
 - 10s - loss: 1.0573 - acc: 0.5935 - val_loss: 1.8868 - val_acc: 0.3917
Epoch 134/250
 - 10s - loss: 1.1041 - acc: 0.5833 - val_loss: 1.8431 - val_acc: 0.3917
Epoch 135/250
 - 10s - loss: 1.1355 - acc: 0.5593 - val_loss: 1.8798 - val_acc: 0.4250
Epoch 136/250
 - 10s - loss: 1.1101 - acc: 0.5759 - val_loss: 1.7061 - val_acc: 0.4667
Epoch 137/250
 - 10s - loss: 1.1096 - acc: 0.5981 - val_loss: 1.9422 - val_acc: 0.4083
Epoch 138/250
 - 10s - loss: 1.0776 - acc: 0.5852 - val_loss: 1.8249 - val_acc: 0.4083
Epoch 139/250
 - 10s - loss: 1.1216 - acc: 0.5704 - val_loss: 1.8520 - val_acc: 0.4167
Epoch 140/250
 - 10s - loss: 1.1278 - acc: 0.5750 - val_loss: 1.8482 - val_acc: 0.4167
Epoch 141/250
 - 10s - loss: 1.0693 - acc: 

...

Epoch 42/250
 - 10s - loss: 1.5591 - acc: 0.3824 - val_loss: 1.7176 - val_acc: 0.3583
Epoch 43/250
 - 10s - loss: 1.5638 - acc: 0.3926 - val_loss: 1.7942 - val_acc: 0.3333
Epoch 44/250
 - 11s - loss: 1.5652 - acc: 0.3880 - val_loss: 1.7493 - val_acc: 0.3417
Epoch 45/250
 - 10s - loss: 1.5689 - acc: 0.3713 - val_loss: 1.7939 - val_acc: 0.3583
Epoch 46/250
 - 10s - loss: 1.5575 - acc: 0.3778 - val_loss: 1.7573 - val_acc: 0.3583
Epoch 47/250
 - 10s - loss: 1.5565 - acc: 0.3852 - val_loss: 1.7455 - val_acc: 0.3667
Epoch 48/250
 - 10s - loss: 1.5655 - acc: 0.3713 - val_loss: 1.8050 - val_acc: 0.3083
Epoch 49/250
 - 10s - loss: 1.5472 - acc: 0.3852 - val_loss: 1.7307 - val_acc: 0.3583
Epoch 50/250
 - 10s - loss: 1.5524 - acc: 0.4009 - val_loss: 1.7415 - val_acc: 0.3417
Epoch 51/250
 - 10s - loss: 1.5392 - acc: 0.3963 - val_loss: 1.7602 - val_acc: 0.3583
Epoch 52/250
 - 10s - loss: 1.5312 - acc: 0.3861 - val_loss: 1.8143 - val_acc: 0.3250
Epoch 53/250
 - 10s - loss: 1.5208 - acc: 0.3907 - val

Epoch 137/250
 - 10s - loss: 1.0549 - acc: 0.6102 - val_loss: 1.6975 - val_acc: 0.4583
Epoch 138/250
 - 10s - loss: 1.0525 - acc: 0.5889 - val_loss: 1.7195 - val_acc: 0.4583
Epoch 139/250
 - 10s - loss: 1.0095 - acc: 0.6259 - val_loss: 1.9565 - val_acc: 0.4000
Epoch 140/250
 - 10s - loss: 1.0194 - acc: 0.6213 - val_loss: 1.6967 - val_acc: 0.4417
Epoch 141/250
 - 10s - loss: 1.0067 - acc: 0.6231 - val_loss: 1.7813 - val_acc: 0.4333
Epoch 142/250
 - 10s - loss: 0.9703 - acc: 0.6269 - val_loss: 1.7975 - val_acc: 0.4583
Epoch 143/250
 - 10s - loss: 1.0071 - acc: 0.6185 - val_loss: 1.6944 - val_acc: 0.4500
Epoch 144/250
 - 10s - loss: 0.9856 - acc: 0.6407 - val_loss: 1.7513 - val_acc: 0.4417
Epoch 145/250
 - 11s - loss: 1.0178 - acc: 0.6231 - val_loss: 1.8506 - val_acc: 0.4417
Epoch 146/250
 - 10s - loss: 1.0120 - acc: 0.6380 - val_loss: 1.6882 - val_acc: 0.4583
Epoch 147/250
 - 10s - loss: 0.9952 - acc: 0.6241 - val_loss: 1.8299 - val_acc: 0.4417
Epoch 148/250
 - 10s - loss: 0.9711 - acc: 

Epoch 232/250
 - 10s - loss: 0.4418 - acc: 0.8444 - val_loss: 2.0930 - val_acc: 0.4417
Epoch 233/250
 - 11s - loss: 0.4584 - acc: 0.8306 - val_loss: 1.8493 - val_acc: 0.4667
Epoch 234/250
 - 10s - loss: 0.4523 - acc: 0.8231 - val_loss: 2.0495 - val_acc: 0.4333
Epoch 235/250
 - 10s - loss: 0.4730 - acc: 0.8380 - val_loss: 1.9139 - val_acc: 0.4167
Epoch 236/250
 - 10s - loss: 0.4517 - acc: 0.8454 - val_loss: 2.1073 - val_acc: 0.4167
Epoch 237/250
 - 10s - loss: 0.4601 - acc: 0.8343 - val_loss: 1.9342 - val_acc: 0.4417
Epoch 238/250
 - 10s - loss: 0.4424 - acc: 0.8491 - val_loss: 1.8596 - val_acc: 0.4583
Epoch 239/250
 - 10s - loss: 0.4120 - acc: 0.8463 - val_loss: 1.8804 - val_acc: 0.4250
Epoch 240/250
 - 10s - loss: 0.4398 - acc: 0.8481 - val_loss: 1.9526 - val_acc: 0.4417
Epoch 241/250
 - 10s - loss: 0.4082 - acc: 0.8509 - val_loss: 2.0356 - val_acc: 0.4583
Epoch 242/250
 - 10s - loss: 0.4849 - acc: 0.8269 - val_loss: 1.8604 - val_acc: 0.4750
Epoch 243/250
 - 10s - loss: 0.4343 - acc: 

Epoch 76/250
 - 10s - loss: 1.4084 - acc: 0.4546 - val_loss: 1.6540 - val_acc: 0.3750
Epoch 77/250
 - 10s - loss: 1.3797 - acc: 0.4648 - val_loss: 1.5405 - val_acc: 0.4333
Epoch 78/250
 - 10s - loss: 1.3798 - acc: 0.4731 - val_loss: 1.6502 - val_acc: 0.3750
Epoch 79/250
 - 10s - loss: 1.3934 - acc: 0.4694 - val_loss: 1.5267 - val_acc: 0.4333
Epoch 80/250
 - 10s - loss: 1.3468 - acc: 0.4741 - val_loss: 1.5941 - val_acc: 0.4000
Epoch 81/250
 - 10s - loss: 1.3447 - acc: 0.4935 - val_loss: 1.5756 - val_acc: 0.4000
Epoch 82/250
 - 10s - loss: 1.3889 - acc: 0.4667 - val_loss: 1.4743 - val_acc: 0.4583
Epoch 83/250
 - 10s - loss: 1.3734 - acc: 0.4704 - val_loss: 1.5079 - val_acc: 0.4833
Epoch 84/250
 - 10s - loss: 1.3541 - acc: 0.4824 - val_loss: 1.5854 - val_acc: 0.3917
Epoch 85/250
 - 10s - loss: 1.3392 - acc: 0.4824 - val_loss: 1.5967 - val_acc: 0.4083
Epoch 86/250
 - 10s - loss: 1.3230 - acc: 0.4991 - val_loss: 1.6700 - val_acc: 0.3667
Epoch 87/250
 - 10s - loss: 1.3563 - acc: 0.4787 - val

Epoch 171/250
 - 10s - loss: 0.8073 - acc: 0.6972 - val_loss: 1.6766 - val_acc: 0.4750
Epoch 172/250
 - 10s - loss: 0.7531 - acc: 0.7333 - val_loss: 1.7732 - val_acc: 0.4583
Epoch 173/250
 - 10s - loss: 0.7803 - acc: 0.7241 - val_loss: 1.7624 - val_acc: 0.4167
Epoch 174/250
 - 10s - loss: 0.8101 - acc: 0.7111 - val_loss: 1.7460 - val_acc: 0.4667
Epoch 175/250
 - 10s - loss: 0.7636 - acc: 0.7120 - val_loss: 1.8755 - val_acc: 0.4333
Epoch 176/250
 - 10s - loss: 0.7863 - acc: 0.7120 - val_loss: 1.8913 - val_acc: 0.3833
Epoch 177/250
 - 10s - loss: 0.7805 - acc: 0.7111 - val_loss: 1.8198 - val_acc: 0.4250
Epoch 178/250
 - 10s - loss: 0.7818 - acc: 0.7074 - val_loss: 1.8343 - val_acc: 0.4417
Epoch 179/250
 - 10s - loss: 0.7602 - acc: 0.7111 - val_loss: 1.8782 - val_acc: 0.4000
Epoch 180/250
 - 10s - loss: 0.7521 - acc: 0.7167 - val_loss: 1.9221 - val_acc: 0.4500
Epoch 181/250
 - 10s - loss: 0.7326 - acc: 0.7333 - val_loss: 1.8439 - val_acc: 0.4667
Epoch 182/250
 - 10s - loss: 0.7366 - acc: 

In [23]:
preds=[models[i].predict(X1) for i in range(6)]

In [26]:
preds=np.array(preds)

In [27]:
preds.shape

(6, 1440, 8)

In [25]:
testaccuracies

[0.3541666666666667,
 0.44583333333333336,
 0.5375,
 0.4166666666666667,
 0.38333333333333336,
 0.42916666666666664]

In [28]:
averagepred=np.mean(preds,axis=0)

In [30]:
averagepred[0]

array([2.0998491e-01, 7.7672213e-01, 1.1461577e-03, 1.0181990e-02,
       2.8624316e-05, 1.2444622e-03, 5.1557174e-04, 1.7618861e-04],
      dtype=float32)

In [31]:
from sklearn.metrics import log_loss

In [32]:
loss=log_loss(ytrain,averagepred)

In [33]:
loss

0.42720193395911743

In [41]:
from sklearn.metrics import precision_recall_curve,f1_score,precision_score,recall_score,accuracy_score
for i in [0.3,0.4,0.41,0.42,0.43,0.44,0.45,0.5,0.6,0.7]:
    pred1=np.array(averagepred>i).astype(np.float32)
    pred1=pred1.reshape((-1,1))
    y=ytrain1.reshape((-1,1))
    print(precision_score(pred1,y))
    print(recall_score(pred1,y))
    print(f1_score(pred1,y))
    print("-------------")

0.9506944444444444
0.8764404609475032
0.9120586275816122
-------------
0.9270833333333334
0.9427966101694916
0.934873949579832
-------------
0.9243055555555556
0.9459843638948117
0.9350193185809625
-------------
0.9229166666666667
0.9520057306590258
0.9372355430183358
-------------
0.9194444444444444
0.9573391178597253
0.9380092100602196
-------------
0.9138888888888889
0.9633967789165446
0.9379900213827512
-------------
0.9055555555555556
0.965925925925926
0.9347670250896057
-------------
0.8611111111111112
0.9779179810725552
0.915805022156573
-------------
0.7715277777777778
0.9901960784313726
0.8672911787665886
-------------
0.6152777777777778
0.9932735426008968
0.7598627787307034
-------------


In [39]:
#we can see 0.43 is best threshold

In [40]:
finalpred=np.array(averagepred>0.43).astype(np.float32)


In [43]:
accuracy_score(finalpred.reshape((-1,1)),ytrain1.reshape((-1,1)))


0.9848090277777778